# Client Integration (TensorBoard)

This example takes TensorBoard's [MNIST MLP example](https://www.tensorflow.org/tensorboard/get_started) and incorportates Verta's Client integration.

In [1]:
HOST = "app.verta.ai"

PROJECT_NAME = "MNIST Multiclassification"
EXPERIMENT_NAME = "FC-NN"

In [2]:
# import os
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 

In [3]:
from verta import Client

client = Client(HOST)
proj = client.set_project(PROJECT_NAME)
expt = client.set_experiment(EXPERIMENT_NAME)

## Imports

In [4]:
from __future__ import print_function

import datetime
import shutil

import tensorflow as tf

---

# Log Workflow

## Prepare Data

In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Define Model

In [6]:
def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

## Run and Log Training

In [7]:
run = client.set_experiment_run()

In [8]:
model = create_model()
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(
    x_train, y_train, 
    epochs=5, 
    validation_data=(x_test, y_test), 
    callbacks=[tensorboard_callback],
)

In [9]:
from verta.integrations.tensorflow import log_tensorboard_events


log_tensorboard_events(run, log_dir)

In [10]:
run

---